In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from drive.MyDrive.resnet import resnet
import cv2

In [3]:
import tensorflow as tf
from tensorflow import keras


In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

In [5]:
X_data = torch.load('/content/drive/MyDrive/CNN/touch_2d_d2_input_tensor.pt')
Y_data = torch.load('/content/drive/MyDrive/CNN/touch_2d_d2_y_train.pt')

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Y_data = LabelEncoder().fit_transform(Y_data)

In [7]:
X_data.shape

torch.Size([408, 3, 512, 512])

In [8]:
target_size = (299, 299)

In [9]:
resized_tensor = F.interpolate(X_data , size=target_size, mode='bilinear', align_corners=False)


In [10]:
X_data=resized_tensor

In [11]:
X_data.shape

torch.Size([408, 3, 299, 299])

In [12]:
import torch
import torchvision.transforms as transforms

# 假设你有原始的数据集 tensor.data 和对应的标签 tensor.labels
data = X_data
labels =torch.tensor(Y_data)

# 计算每个标签的样本数量
label_counts = torch.bincount(labels)

# 计算样本数量最多的标签的样本数量
max_count = label_counts.max().item()

# 定义数据扩增的变换
transform = transforms.Compose([
    # 添加你需要的数据扩增操作，比如随机裁剪、翻转等
    transforms.RandomCrop(299),
    transforms.RandomHorizontalFlip(),
    # 其他数据扩增操作...
])

# 创建一个空列表，用于存储扩增后的数据和标签
augmented_data = []
augmented_labels = []

# 遍历每个标签
for label in torch.unique(labels):
    # 获取该标签下的索引
    indices = torch.nonzero(labels == label).squeeze()

    # 计算需要扩增的样本数量
    num_augmented_samples = max_count - label_counts[label]

    # 随机选择需要扩增的样本索引
    selected_indices = torch.randint(low=0, high=len(indices), size=(num_augmented_samples,))

    # 对选中的样本进行数据扩增操作，并将扩增后的数据和标签添加到列表中
    for index in selected_indices:
        image = data[indices[index]]
        augmented_image = transform(image)
        augmented_data.append(augmented_image)
        augmented_labels.append(label)

# 将扩增后的数据和标签转换为张量
augmented_data = torch.stack(augmented_data)
augmented_labels = torch.tensor(augmented_labels)

# 将原始数据和扩增后的数据合并
final_data = torch.cat((data, augmented_data))
final_labels = torch.cat((labels, augmented_labels))


In [13]:
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final_data, final_labels, stratify=final_labels, test_size=0.2, random_state=42)


In [14]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]

        if self.transform is not None:
            image = self.transform(image)
            label = self.transform(label)

        return image, label
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 标准化
])





In [15]:
train_dataset=CustomDataset(x_train, y_train)
test_dataset=CustomDataset(x_test, y_test )

# 定义数据加载器
shuffle = True



In [16]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

In [17]:
import torch.nn as nn
import torchvision.models as models

In [19]:
modelnet = models.inception_v3(pretrained=True)

In [ ]:
from torchvision.utils import make_grid
for images, labels in train_loader:
    print('Images Shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

In [21]:
num_epochs = 100
learning_rate = 0.001

In [25]:
# 创建Net网络模型
if torch.cuda.is_available():
   modelnet.cuda()
device = torch.device("cuda")
#resnet.cuda()
# 获取Net模型的最后一个全连接层
num_ftrs = modelnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)
last_fully_connected_layer = modelnet.fc

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelnet.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将数据移动到GPU
        if torch.cuda.is_available():
            resnet = modelnet.to(device)
            images = images.cuda()
            labels = labels.cuda()

        # 前向传递，反向传递，优化器更新
        outputs = modelnet(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 输出训练信息
        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))
# 提取最后一个全连接层
last_fc_layer_weights = last_fully_connected_layer.weight
last_fc_layer_biases = last_fully_connected_layer.bias

# 打印最后一个全连接层的权重和偏置向量的形状
print(last_fc_layer_weights.shape) 
print(last_fc_layer_biases.shape)  

torch.Size([2, 2048])
torch.Size([2])


In [24]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据移动到GPU
        if torch.cuda.is_available():
            resnet = modelnet.to(device)
            images = images.cuda()
            labels = labels.cuda()

        # 前向传递，并获取最大预测值的索引
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)

        # 统计正确预测的样本数和总样本数
        total += labels.size(0)
        correct += (predicted.cpu() == labels.cpu()).sum().item()

# 打印验证集的精度
print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Accuracy of the model on the test images: 90 %


In [57]:
#截取全链接层
from torchsummary import summary

newmodel = torch.nn.Sequential(*(list(modelnet.children())[:-1]))
summary(newmodel, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           9,408
       BatchNorm2d-2         [-1, 64, 150, 150]             128
              ReLU-3         [-1, 64, 150, 150]               0
         MaxPool2d-4           [-1, 64, 75, 75]               0
            Conv2d-5           [-1, 64, 75, 75]           4,096
       BatchNorm2d-6           [-1, 64, 75, 75]             128
              ReLU-7           [-1, 64, 75, 75]               0
            Conv2d-8           [-1, 64, 75, 75]          36,864
       BatchNorm2d-9           [-1, 64, 75, 75]             128
             ReLU-10           [-1, 64, 75, 75]               0
           Conv2d-11          [-1, 256, 75, 75]          16,384
      BatchNorm2d-12          [-1, 256, 75, 75]             512
           Conv2d-13          [-1, 256, 75, 75]          16,384
      BatchNorm2d-14          [-1, 256,

In [59]:
model_state_dict = newmodel.state_dict()

# 指定保存路径和文件名
save_path = 'model.pth'

# 将模型状态字典保存到.pth文件
torch.save(model_state_dict, save_path)
